# This book is identical to RNN_corn_daily_Wang.ipynb, except for the generation of test set predictions after validation set predictions

### CONTENTS
### 1 Imports
### 2 Arrange datasets
### 3 Define models
### 4 Train models

# 1 Imports

In [1]:
# RNN imports

import tensorflow as tf

from keras.preprocessing.sequence import TimeseriesGenerator

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN

from keras.models import load_model

from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
#General imports

import numpy as np
import pandas as pd

import scipy.stats as stats

import matplotlib.pyplot as plt
import matplotlib.dates

from datetime import datetime

from google.colab import drive
import json

In [22]:
# Import daily data

data_url = 'https://raw.githubusercontent.com/ptraver/data/main/corn_daily_V2.csv'
corn_df = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='date')

# fix bad name
corn_df.columns = ['value']

# take only 1974 - 2017
corn_df = corn_df['1974-01-01':'2017-12-31']

# corn_df needs to lose last value becuase during process of VMD, the last value is lost
corn_df = corn_df[:-1]

# view loaded data
corn_df.tail(2)

,value
date,
2017-12-20,349.25
2017-12-21,350.75


In [23]:
# Import emd data
data_url = 'https://raw.githubusercontent.com/ptraver/data/main/corn_emd_daily.csv'
corn_emd = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='date')

In [24]:
# Import vmd data
data_url = 'https://raw.githubusercontent.com/ptraver/data/main/corn_vmd_daily.csv'
corn_vmd = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='date')

# 2 Arrange datasets

In [25]:
# Split data so that valid and test do not feature in decisions
from sklearn.model_selection import train_test_split

true_train, out_of_sample = train_test_split(corn_df, test_size=0.2, shuffle=False)
true_valid, true_test = train_test_split(out_of_sample, test_size=0.5, shuffle=False)

emd_train, out_of_sample = train_test_split(corn_emd, test_size=0.2, shuffle=False)
emd_valid, emd_test = train_test_split(out_of_sample, test_size=0.5, shuffle=False)

vmd_train, out_of_sample = train_test_split(corn_vmd, test_size=0.2, shuffle=False)
vmd_valid, vmd_test = train_test_split(out_of_sample, test_size=0.5, shuffle=False)

In [26]:
# Scale data between 0 and 1
from sklearn.preprocessing import MinMaxScaler

emd_scaler = MinMaxScaler()
emd_train = emd_scaler.fit_transform(emd_train)
emd_valid = emd_scaler.transform(emd_valid)
emd_test = emd_scaler.transform(emd_test)

vmd_scaler = MinMaxScaler()
vmd_train = vmd_scaler.fit_transform(vmd_train)
vmd_valid = vmd_scaler.transform(vmd_valid)
vmd_test = vmd_scaler.transform(vmd_test)

true_scaler = MinMaxScaler()
true_train = true_scaler.fit_transform(true_train)
true_valid = true_scaler.transform(true_valid)
true_test = true_scaler.transform(true_test)

In [8]:
# Set input shape variables
n_inputs = 60
n_features = 1

In [27]:
# get emd sets in order
emd_extra_valid = emd_train[-n_inputs:]
emd_valid_set = np.concatenate((emd_extra_valid, emd_valid))

emd_extra_test = emd_valid[-n_inputs:]
emd_test_set = np.concatenate((emd_extra_test, emd_test))

emd_train_shaped = TimeseriesGenerator(emd_train, emd_train, length=n_inputs, batch_size=1)
emd_valid_shaped = TimeseriesGenerator(emd_valid_set, emd_valid_set, length=n_inputs, batch_size=1)
emd_test_shaped = TimeseriesGenerator(emd_test_set, emd_test_set, length=n_inputs, batch_size=1)

In [28]:
# get vmd sets in order
vmd_extra_valid = vmd_train[-n_inputs:]
vmd_valid_set = np.concatenate((vmd_extra_valid, vmd_valid))

vmd_extra_test = vmd_valid[-n_inputs:]
vmd_test_set = np.concatenate((vmd_extra_test, vmd_test))

vmd_train_shaped = TimeseriesGenerator(vmd_train, vmd_train, length=n_inputs, batch_size=1)
vmd_valid_shaped = TimeseriesGenerator(vmd_valid_set, vmd_valid_set, length=n_inputs, batch_size=1)
vmd_test_shaped = TimeseriesGenerator(vmd_test_set, vmd_test_set, length=n_inputs, batch_size=1)

In [29]:
# get non-decomposed sets in order
extra_valid = true_train[-n_inputs:]
valid_set = np.concatenate((extra_valid, true_valid))

extra_test = true_valid[-n_inputs:]
test_set = np.concatenate((extra_test, true_test))

train_shaped = TimeseriesGenerator(true_train, true_train, length=n_inputs, batch_size=1)
valid_shaped = TimeseriesGenerator(valid_set, valid_set, length=n_inputs, batch_size=1)
test_shaped = TimeseriesGenerator(test_set, test_set, length=n_inputs, batch_size=1)

# 3 Define models

In [ ]:
# Set variables

units = 32

In [ ]:
# Define model definition function

def define_model():
  RNNregressor = Sequential()
  RNNregressor.add(SimpleRNN(units=units, input_shape=(n_inputs, n_features)))
  RNNregressor.add(Dense(units=1))
  RNNregressor.compile(optimizer='adam', loss='mse')
  return RNNregressor

# 4 Train models

In [ ]:
# Set variables

EPOCHS = 4

In [12]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###on EMD corn daily

In [ ]:
# Set checkpoint callback
filepath = 'emd_RNN_daily_valid_{epoch:02d}'
checkpoint = ModelCheckpoint('models/{}.model'.format(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min'))

# Define model
EMDregressor = define_model()

# Train model
with tf.device('/device:GPU:0'):
  EMDregressor.fit(emd_train_shaped,
                   epochs=EPOCHS,
                   validation_data=emd_valid_shaped,
                   callbacks=[checkpoint])

Epoch 1/4
8807/8807 [==============================] - 403s 46ms/step - loss: 1.3597e-04 - val_loss: 0.0015
Epoch 2/4
8807/8807 [==============================] - 393s 45ms/step - loss: 2.0355e-05 - val_loss: 3.9755e-05
Epoch 3/4
8807/8807 [==============================] - 393s 45ms/step - loss: 4.6570e-05 - val_loss: 3.7826e-05
Epoch 4/4
8807/8807 [==============================] - 397s 45ms/step - loss: 6.5557e-06 - val_loss: 1.3389e-04


In [50]:
## ACTION REQUIRED ##

# Select best model by lowest validation loss
best_model = 'emd_RNN_daily_valid_03'

In [ ]:
# Save best model to Drive
emd_RNN_daily_model = load_model(f'models/{best_model}.model')
emd_RNN_daily_model.save(f'/content/gdrive/My Drive/models/{best_model}.h5')

In [ ]:
# Use best model to generate emd validation set forecasts
emd_fc = emd_RNN_daily_model.predict(emd_valid_shaped)

In [ ]:
# Rescale forecasted values
emd_rescaled = emd_scaler.inverse_transform(emd_fc)

In [ ]:
# Save emd validation set predictions
np.save('/content/gdrive/My Drive/outputs/emd_RNN_daily_valid.npy', emd_rescaled.flatten())

In [ ]:
#### TEST SET ####

In [51]:
# Load the model
emd_RNN_daily_model = load_model(f'/content/gdrive/My Drive/models/{best_model}.h5')

In [52]:
# Use best model to generate emd test set forecasts
emd_fc = emd_RNN_daily_model.predict(emd_test_shaped)

In [53]:
# Rescale forecasted values
emd_rescaled = emd_scaler.inverse_transform(emd_fc)

In [54]:
# Save emd validation set predictions
np.save('/content/gdrive/My Drive/outputs/emd_RNN_daily_test.npy', emd_rescaled.flatten())

###on VMD corn daily

In [ ]:
# Set checkpoint callback
filepath = 'vmd_RNN_daily_valid_{epoch:02d}'
checkpoint = ModelCheckpoint('models/{}.model'.format(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min'))

# Define model
VMDregressor = define_model()

# Train model
with tf.device('/device:GPU:0'):
  VMDregressor.fit(vmd_train_shaped,
                   epochs=EPOCHS,
                   validation_data=vmd_valid_shaped,
                   callbacks=[checkpoint])

Epoch 1/4
8807/8807 [==============================] - 395s 45ms/step - loss: 2.8027e-04 - val_loss: 1.9152e-04
Epoch 2/4
8807/8807 [==============================] - 397s 45ms/step - loss: 2.9085e-05 - val_loss: 5.2343e-05
Epoch 3/4
8807/8807 [==============================] - 399s 45ms/step - loss: 1.8044e-05 - val_loss: 3.5807e-04
Epoch 4/4
8807/8807 [==============================] - 400s 45ms/step - loss: 1.3643e-05 - val_loss: 2.6376e-04


In [35]:
## ACTION REQUIRED ##

# Select best model by lowest validation loss
best_model = 'vmd_RNN_daily_valid_02'

In [ ]:
# Save best model to Drive
vmd_RNN_daily_model = load_model(f'models/{best_model}.model')
vmd_RNN_daily_model.save(f'/content/gdrive/My Drive/models/{best_model}.h5')

In [ ]:
# Use best model to generate emd validation set forecasts
vmd_fc = vmd_RNN_daily_model.predict(vmd_valid_shaped)

In [ ]:
# Rescale forecasted values
vmd_rescaled = vmd_scaler.inverse_transform(vmd_fc)

In [ ]:
# Save emd validation set predictions
np.save('/content/gdrive/My Drive/outputs/vmd_RNN_daily_valid.npy', vmd_rescaled.flatten())

In [ ]:
# Use best model to generate emd validation set forecasts
vmd_fc = vmd_RNN_daily_model.predict(vmd_test_shaped)

In [ ]:
# Rescale forecasted values
vmd_rescaled = vmd_scaler.inverse_transform(vmd_fc)

In [ ]:
# Save emd validation set predictions
np.save('/content/gdrive/My Drive/outputs/vmd_RNN_daily_test.npy', vmd_rescaled.flatten())

In [36]:
#### TEST SET ####

In [37]:
# Load the model
vmd_RNN_daily_model = load_model(f'/content/gdrive/My Drive/models/{best_model}.h5')

In [38]:
# Use best model to generate emd test set forecasts
vmd_fc = vmd_RNN_daily_model.predict(vmd_test_shaped)

In [39]:
# Rescale forecasted values
vmd_rescaled = vmd_scaler.inverse_transform(vmd_fc)

In [41]:
# Save emd validation set predictions
np.save('/content/gdrive/My Drive/outputs/vmd_RNN_daily_test.npy', vmd_rescaled.flatten())

## on non-decomposed

In [ ]:
# Set checkpoint callback
filepath = 'true_RNN_daily_valid_{epoch:02d}'
checkpoint = ModelCheckpoint('models/{}.model'.format(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min'))

# Define model
TRUEregressor = define_model()

# Train model
with tf.device('/device:GPU:0'):
  TRUEregressor.fit(train_shaped,
                   epochs=EPOCHS,
                   validation_data=valid_shaped,
                   callbacks=[checkpoint])

Epoch 1/4
8807/8807 [==============================] - 397s 45ms/step - loss: 6.7504e-04 - val_loss: 6.8697e-04
Epoch 2/4
8807/8807 [==============================] - 393s 45ms/step - loss: 1.0797e-04 - val_loss: 6.3847e-04
Epoch 3/4
8807/8807 [==============================] - 426s 48ms/step - loss: 9.9561e-05 - val_loss: 6.3956e-04
Epoch 4/4
8807/8807 [==============================] - 392s 44ms/step - loss: 9.6490e-05 - val_loss: 9.1771e-04


In [42]:
## ACTION REQUIRED ##

# Select best model by lowest validation loss
best_model = 'true_RNN_daily_valid_02'

In [ ]:
# Save best model to Drive
true_RNN_daily_model = load_model(f'models/{best_model}.model')
true_RNN_daily_model.save(f'/content/gdrive/My Drive/models/{best_model}.h5')

In [ ]:
# Use best model to generate emd validation set forecasts
true_fc = true_RNN_daily_model.predict(valid_shaped)

In [ ]:
# Rescale forecasted values
true_rescaled = true_scaler.inverse_transform(true_fc)

In [ ]:
# Save emd validation set predictions
np.save('/content/gdrive/My Drive/outputs/true_RNN_daily_valid.npy', true_rescaled.flatten())

In [ ]:
# Use best model to generate emd validation set forecasts
true_fc = true_RNN_daily_model.predict(test_shaped)

In [ ]:
# Rescale forecasted values
true_rescaled = true_scaler.inverse_transform(true_fc)

In [ ]:
# Save emd validation set predictions
np.save('/content/gdrive/My Drive/outputs/true_RNN_daily_test.npy', true_rescaled.flatten())

In [43]:
#### TEST SET ####

In [44]:
# Load the model
true_RNN_daily_model = load_model(f'/content/gdrive/My Drive/models/{best_model}.h5')

In [46]:
# Use best model to generate emd test set forecasts
true_fc = true_RNN_daily_model.predict(test_shaped)

In [47]:
# Rescale forecasted values
true_rescaled = true_scaler.inverse_transform(true_fc)

In [49]:
# Save emd validation set predictions
np.save('/content/gdrive/My Drive/outputs/true_RNN_daily_test.npy', true_rescaled.flatten())

# End